## Preprocessing

In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [18]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [20]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [21]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [22]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [23]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classified_counts = application_df['CLASSIFICATION'].value_counts()
filtered_classified = classified_counts[classified_counts > 1]
filtered_classified

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [24]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1883].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [25]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS','IS_SUCCESSFUL'])

In [39]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns='IS_SUCCESSFUL_1')  # Features

y = application_df['IS_SUCCESSFUL_1']  # Target variable

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [44]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=45))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 276       
                                                                 
 dense_4 (Dense)             (None, 6)                 42        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 325 (1.27 KB)
Trainable params: 325 (1.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [46]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200


858/858 [==============================] - 2s 1ms/step - loss: 0.2826 - accuracy: 0.8713
Epoch 2/200
858/858 [==============================] - 1s 1ms/step - loss: 0.0192 - accuracy: 0.9979
Epoch 3/200
858/858 [==============================] - 1s 973us/step - loss: 0.0063 - accuracy: 0.9992
Epoch 4/200
858/858 [==============================] - 1s 983us/step - loss: 0.0029 - accuracy: 0.9996
Epoch 5/200
858/858 [==============================] - 1s 965us/step - loss: 0.0015 - accuracy: 0.9998
Epoch 6/200
858/858 [==============================] - 1s 1ms/step - loss: 6.7097e-04 - accuracy: 0.9999
Epoch 7/200
858/858 [==============================] - 1s 956us/step - loss: 5.4693e-04 - accuracy: 0.9999
Epoch 8/200
858/858 [==============================] - 1s 956us/step - loss: 1.5119e-04 - accuracy: 1.0000
Epoch 9/200
858/858 [==============================] - 1s 1ms/step - loss: 1.0190e-04 - accuracy: 1.0000
Epoch 10/200
858/858 [==============================] - 1s 980u

In [47]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.0020 - accuracy: 0.9999 - 308ms/epoch - 1ms/step
Loss: 0.002029044320806861, Accuracy: 0.9998542070388794


In [48]:
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=300)

Epoch 1/300
858/858 [==============================] - 1s 986us/step - loss: 5.3448e-11 - accuracy: 1.0000
Epoch 2/300
858/858 [==============================] - 1s 956us/step - loss: 5.3207e-11 - accuracy: 1.0000
Epoch 3/300
858/858 [==============================] - 1s 1ms/step - loss: 5.2957e-11 - accuracy: 1.0000
Epoch 4/300
858/858 [==============================] - 1s 1ms/step - loss: 5.2719e-11 - accuracy: 1.0000
Epoch 5/300
858/858 [==============================] - 1s 1ms/step - loss: 5.2505e-11 - accuracy: 1.0000
Epoch 6/300
858/858 [==============================] - 1s 1ms/step - loss: 5.2294e-11 - accuracy: 1.0000
Epoch 7/300
858/858 [==============================] - 1s 1ms/step - loss: 5.2077e-11 - accuracy: 1.0000
Epoch 8/300
858/858 [==============================] - 1s 979us/step - loss: 5.1845e-11 - accuracy: 1.0000
Epoch 9/300
858/858 [==============================] - 1s 1ms/step - loss: 5.1637e-11 - accuracy: 1.0000
Epoch 10/300
858/858 [===========================

In [49]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.0021 - accuracy: 0.9999 - 181ms/epoch - 840us/step
Loss: 0.0021055121906101704, Accuracy: 0.9998542070388794


In [50]:
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
858/858 [==============================] - 1s 1ms/step - loss: 3.5530e-11 - accuracy: 1.0000
Epoch 2/100
858/858 [==============================] - 1s 1ms/step - loss: 3.5541e-11 - accuracy: 1.0000
Epoch 3/100
858/858 [==============================] - 1s 1ms/step - loss: 3.5546e-11 - accuracy: 1.0000
Epoch 4/100
858/858 [==============================] - 1s 1ms/step - loss: 3.5549e-11 - accuracy: 1.0000
Epoch 5/100
858/858 [==============================] - 1s 982us/step - loss: 3.5558e-11 - accuracy: 1.0000
Epoch 6/100
858/858 [==============================] - 1s 992us/step - loss: 3.5567e-11 - accuracy: 1.0000
Epoch 7/100
858/858 [==============================] - 1s 969us/step - loss: 3.5575e-11 - accuracy: 1.0000
Epoch 8/100
858/858 [==============================] - 1s 956us/step - loss: 3.5584e-11 - accuracy: 1.0000
Epoch 9/100
858/858 [==============================] - 1s 1ms/step - loss: 3.5589e-11 - accuracy: 1.0000
Epoch 10/100
858/858 [=========================

In [51]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.0021 - accuracy: 0.9999 - 214ms/epoch - 996us/step
Loss: 0.0021363014820963144, Accuracy: 0.9998542070388794


In [53]:
# Export our model to HDF5 file
nn_model.save('nn_model.h5')

c:\Users\watsu\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
